In [1]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
# from datetime import datetime
import sqlite3
from sqlalchemy import create_engine

In [2]:
import sys
import logging
path = r"..\..\Logs\clinical_log.log"
logging.basicConfig(filename=path,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d,%(name)s,%(levelname)s,%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger("WorkedOnClaims")

In [3]:
conn = create_engine(r'mssql+pyodbc://@PYTHONSERVER\SQLEXPRESS/InSync?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes', fast_executemany=True)

# To do
Need to fix merge so that it uses a combination of claim number and line-item

In [4]:
try:
    logger.info("Creating a dataframe from tblPayment table as well as other tables ")
    sql = '''
    SELECT
        ClaimNo,
        tblPayment.CreatedOn,
        tblPayment.CreatedBy,
        tblResourceDetails1.FullName,
        DOSFrom
    FROM 
        tblPayment
        LEFT JOIN tblChargeDetails ON (tblChargeDetails.ChargeID=tblPayment.ClaimNo)
        LEFT JOIN tblUser ON (tblPayment.CreatedBy=tblUser.UserID)
        LEFT JOIN 
        (SELECT   
            DISTINCT(UserID), 
            FullName
        FROM 
            tblResourceDetails) 
        tblResourceDetails1 ON (tblUser.Userid=tblResourceDetails1.Userid)
    '''
    payment_df = pd.read_sql(sql, conn)
    logger.info("Successfully created a dataframe from tblPayment table")
    payment_df['CreatedOn'] = pd.to_datetime(payment_df['CreatedOn'])
    payment_df['DOSFrom'] = pd.to_datetime(payment_df['DOSFrom'])
    # in order to merge the payment and submissions dataframes on the claimNo, we need to make sure they're sorted the same 
    # otherwise createdon dates and submission dates won't match up 
    payment_df = payment_df.sort_values(by=['ClaimNo', 'CreatedOn'], ascending=[True, False])
except Exception as e:
    print(e)
    logger.error("Failed to create a dataframe from tblPayment table")     

In [5]:
try:
    logger.info("Creating a dataframe from tblClaimSubmission")
    sql = '''
    SELECT
        ClaimNumber as ClaimNo,
        SubmissionDate
    FROM 
        tblClaimSubmission
    '''
    submissions = pd.read_sql(sql, conn)
    logger.info("Successfully created a dataframe from tblClaimSubmission")
    submissions['SubmissionDate'] = pd.to_datetime(submissions['SubmissionDate'])
    # in order to merge the payment and submissions dataframes on the claimNo, we need to make sure they're sorted the same 
    # otherwise createdon dates and submission dates won't match up 
    submissions = submissions.sort_values(by=['ClaimNo', 'SubmissionDate'], ascending=[True, False])
except Exception as e:
    print(e)
    logger.error("Failed to create a dataframe from tblClaimSubmission table")

In [6]:
try:
    logger.info("Merging payment_df and submissions dataframes")
    payment_df = payment_df.merge(submissions, how='left', on='ClaimNo', validate='m:m')
    logger.info("Successfully merged payment_df and submissions dataframes")
except Exception as e:
    print(e)
    logger.error("Failed to merge payment_df and submissions dataframes")    

In [7]:
try:
    logger.info("Creating a dataframe from tblPaymentDetails table and tblPaymentTransactionTypes table")
    sql = '''
    SELECT
        ClaimNo,
        DisplayName as HadDenial
    FROM 
        tblPaymentDetails
        LEFT JOIN tblPaymentTransactionTypes ON (tblPaymentDetails.TransactionType=tblPaymentTransactionTypes.TransactionType)
    WHERE 
        DisplayName = 'Denial'
    '''
    denials = pd.read_sql(sql, conn)
    logger.info("Successfully created a dataframe from tblPaymentDetails table and tblPaymentTransactionTypes table")
    denials = denials.drop_duplicates('ClaimNo')
except Exception as e:
    print(e)
    logger.error("Failed to create a dataframe from tblPaymentDetails table and tblPaymentTransactionTypes table")

In [8]:
try:
    logger.info("Filtering payment_df for payments that have a denial")
    # did payment have a denial
    payment_df = payment_df[payment_df["ClaimNo"].isin(denials['ClaimNo'])].copy()
    logger.info("Successfully filtered payment_df for payments that have a denial")
except Exception as e:
    print(e)
    logger.error("Failed to filter payment_df for payments that have a denial")

The below code could be improved with a groupby statement

In [9]:
try:
    # Filtering payment_df for payments that have a denial
    # Denials that weren’t worked on have a submission date prior to their most recent entered on date and the created by user is not the system (user id -99)
    payment_df.sort_values(by='SubmissionDate', inplace=True, ascending=False)
    # Most recent Entered On 
    mostRecentPayment = payment_df.drop_duplicates("ClaimNo", keep='first').copy()
    mostRecentPayment['WorkedOn'] = (mostRecentPayment['SubmissionDate'] > mostRecentPayment['CreatedOn']) & (mostRecentPayment['CreatedBy'] != -99)
    # mostRecentPayment['WorkedOn'] = mostRecentPayment.apply(lambda row: False if row['CreatedBy'] == -99 else row['WorkedOn'], axis=1)
    logger.info("Successfully filtered payment_df for payments that have a denial")
except Exception as e:
    print(e)
    logger.error("Failed to Filter payment_df for payments that have a denial")

In [10]:
try:
    logger.info("Creating a new column NeedsWork for claims that need to be worked on")
    # They need to be worked on if they haven't yet been worked on. So "needs to be" is the inverse of WorkedOn
    logger.info("Successfully created a new column NeedsWork for claims that need to be worked on")
    mostRecentPayment['NeedsWork'] = ~mostRecentPayment['WorkedOn']
except Exception as e:
    print(e)    
    logger.error("Failed to create a new column NeedsWork for claims that need to be worked on")    

In [11]:
try:
    logger.info("Adding/replacing columns in ptWorkedOnDenials with columns from workedOnDF dataframe")
    workedOnDf = mostRecentPayment[['ClaimNo', 'NeedsWork']].copy()
    workedOnDf['NeedsWork'] = workedOnDf['NeedsWork'].astype(str)
    workedOnDf['ClaimNo'] = workedOnDf['ClaimNo'].astype(str)
    workedOnDf.to_sql("ptWorkedOnDenials",conn, if_exists="replace", index=False)
    logger.info("Success in Adding/replacing columns in ptWorkedOnDenials with columns from workedOnDF dataframe")
except Exception as e:
    print(e)
    logger.error("Failed in Adding/replacing columns in ptWorkedOnDenials with columns from workedOnDF dataframe")